## All coding work has been done on colab for the simple reason that I do not have an SSD on my PC.

I have used Wav2Lip package from IIIT Hyderabad toperform the lip sync task. I have first installed all the dependancies and imported a lot of packages that I have used for this project   

# Installing Dependencies and Packages


In [ ]:
!rm -rf /content/sample_data
!mkdir /content/sample_data

!git clone https://github.com/justinjohn0306/Wav2Lip

#download the pretrained model
!wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O '/content/Wav2Lip/checkpoints/wav2lip_gan.pth'
a = !pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl


# !pip uninstall tensorflow tensorflow-gpu
!cd Wav2Lip && pip install -r requirements.txt

#download pretrained model for face detection
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "/content/Wav2Lip/face_detection/detection/sfd/s3fd.pth"

!pip install ffmpeg-python

In [2]:
from base64 import b64decode
from base64 import b64encode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

import os
import shutil
from google.colab import drive
from google.colab import files
import moviepy.editor as mp
from moviepy.editor import VideoFileClip

from IPython.display import HTML

from IPython.display import Audio
from IPython.core.display import display

from moviepy.editor import VideoFileClip
from IPython.display import clear_output
from base64 import b64decode


# Uploading video into colab

The video was first uploaded into colab and then it's path was given as input below. I have converted the video into a lower resolution format because the Wave2Lip model was trained on a low resolution dataset.  

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
def get_resolution(video_path):
    video = VideoFileClip(video_path)
    return video.size


def resize_video_to_720p(input_path, output_path):
    clip = VideoFileClip(input_path)
    resized_clip = clip.resize(height=720)
    resized_clip.write_videofile(output_path, codec="libx264", audio_codec="aac")

def showVideo(path):
  mp4 = open(str(path),'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML("""
  <video width=700 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url)

# taking in the video file path .
video_path = str(input("Input the path to your video file"))

resolution = get_resolution(video_path)
print("The resolution of the given video :", resolution)

# I have saved the video file into /content/sample_data/input_vid.mp4 because it
# becomes easier to work with the file. Sometimes the model does not consider
#the video file if there are any blank spaces in it

if resolution[0] > 1280 or resolution[1] > 720:
  print("The model works well with low resolution videos. Hence I will be resizing the video ")
  output_video_path = '/content/sample_data/input_vid.mp4'
  resize_video_to_720p(video_path, output_video_path)
  video_path = "/content/sample_data/input_vid.mp4"

else:
    print("No resizing needed")


Input the path to video file/content/My_Movie _15.mp4
The resolution of the given video : [1920, 1080]
The model works well with low resolution videos. Hence I will be resizing the video 
Moviepy - Building video /content/sample_data/input_vid.mp4.
MoviePy - Writing audio in input_vidTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/sample_data/input_vid.mp4



Moviepy - Done !
Moviepy - video ready /content/sample_data/input_vid.mp4


## Uploading the audio file and then storing it in a preferred location

In [6]:
AUDIO_path = str(input("Input path to audio file"))

# Load audio with specified sampling rate
import librosa
audio, sr = librosa.load(AUDIO_path, sr=None)

import soundfile as sf
sf.write('/content/sample_data/input_audio.wav', audio, sr, format='wav')

Input path to audio file/content/15sec_audio.wav


# This is the final code that trains the model.

There are only a few parameters that can be altered while training and I have experimented with them. I have montioned the best values for each of the parameters that I found.

In [7]:
pad_top =  15#@param {type:"integer"}
pad_bottom =  5#@param {type:"integer"}
pad_left =  -6#@param {type:"integer"}
pad_right =  10#@param {type:"integer"}
rescaleFactor =  1#@param {type:"integer"}
nosmooth = False #@param {type:"boolean"}

if nosmooth == False:
  !cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "../sample_data/input_vid.mp4" --audio "../sample_data/input_audio.wav" --pads $pad_top $pad_bottom $pad_left $pad_right --resize_factor $rescaleFactor
else:
  !cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "../sample_data/input_vid.mp4" --audio "../sample_data/input_audio.wav" --pads $pad_top $pad_bottom $pad_left $pad_right --resize_factor $rescaleFactor --nosmooth


Using cuda for inference.
Reading video frames...
Number of frames available for inference: 384
(80, 1121)
Length of mel chunks: 347
  0% 0/3 [00:00<?, ?it/s]
  0% 0/22 [00:00<?, ?it/s]
  5% 1/22 [03:13<1:07:34, 193.08s/it]
  9% 2/22 [03:16<27:07, 81.36s/it]   
 14% 3/22 [03:19<14:24, 45.51s/it]
 18% 4/22 [03:22<08:36, 28.71s/it]
 23% 5/22 [03:25<05:32, 19.55s/it]
 27% 6/22 [03:28<03:42, 13.88s/it]
 32% 7/22 [03:31<02:34, 10.28s/it]
 36% 8/22 [03:33<01:50,  7.92s/it]
 41% 9/22 [03:36<01:22,  6.37s/it]
 45% 10/22 [03:40<01:04,  5.37s/it]
 50% 11/22 [03:42<00:50,  4.62s/it]
 55% 12/22 [03:46<00:41,  4.17s/it]
 59% 13/22 [03:48<00:33,  3.77s/it]
 64% 14/22 [03:52<00:28,  3.57s/it]
 68% 15/22 [03:55<00:24,  3.44s/it]
 73% 16/22 [03:58<00:19,  3.26s/it]
 77% 17/22 [04:00<00:15,  3.14s/it]
 82% 18/22 [04:03<00:12,  3.09s/it]
 86% 19/22 [04:07<00:09,  3.13s/it]
 91% 20/22 [04:09<00:06,  3.05s/it]
 95% 21/22 [04:12<00:02,  3.00s/it]
100% 22/22 [04:40<00:00, 12.75s/it]
Load checkpoint from: che

## Code for final video preview

In [ ]:
print("Final Video Preview")
print("Download this video from", '/content/Wav2Lip/results/result_voice.mp4')
showVideo('/content/Wav2Lip/results/result_voice.mp4')